# Self-Driving Car Engineer Nanodegree


## Project 1: **Finding Lane Lines on the Road** 


In [1]:
# import everything we'll need

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import cv2
import math
import os
from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

In [2]:
# define key parameters

output_dir = 'test_images_output/'
test_image_dir = 'test_images/'

gaussian_kernel_size = 5
canny_low = 50
canny_high = 150
roi_h = 0.6
roi_w1 = 0.47
roi_w2 = 0.53
hough_rho = 1
hough_theta = np.pi/180
hough_threshold = 32
hough_min_len = 16
hough_min_gap = 16
slope_high = 1.0
slope_low = 0.5

In [3]:
# define helper functions

def grayscale(img):
    # Applies the Grayscale transform
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    
def canny(img, low_threshold, high_threshold):
    # Applies the Canny transform
    return cv2.Canny(img, low_threshold, high_threshold)

def gaussian_blur(img, kernel_size):
    # Applies a Gaussian Noise kernel
    return cv2.GaussianBlur(img, (kernel_size, kernel_size), 0)

def region_of_interest(img, vertices):
    # Applies an image mask
    
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image


def draw_lines(img, lines, color=[255, 0, 0], thickness=2):    # note: OpenCV
    # Draws lines on an image. 
    # Note that color is defined as BRG in OpenCV, so need to modify if using OpenCV image open/write.
    
    for line in lines:
        for x1,y1,x2,y2 in line:
            cv2.line(img, (x1, y1), (x2, y2), color, thickness)

def hough_lines(img, rho, theta, threshold, min_line_len, max_line_gap):
    # Returns an image with hough lines drawn.

    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), minLineLength=min_line_len, maxLineGap=max_line_gap)
    line_img = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
    draw_lines(line_img, lines)
    return line_img

def weighted_img(img, initial_img, α=0.8, β=1., λ=0.):
    # Returns blended image as: initial_img * α + img * β + λ
    
    return cv2.addWeighted(initial_img, α, img, β, λ)

def hough_lines_extended(img, rho, theta, threshold, min_line_len, max_line_gap, slope_high, slope_low):
    # Returns an image with extended lanes lines drawn based on average of hough lines

    # set up
    y_max,x_max,_ = im.shape
    left_slopes = []
    left_intercepts = []
    right_slopes= []
    right_intercepts = []

    # detect lines
    lines = cv2.HoughLinesP(img, rho, theta, threshold, np.array([]), 
                            minLineLength=min_line_len, maxLineGap=max_line_gap)
 
    for line in lines:
        for x1,y1,x2,y2 in line:
            
            # calculate slope and y-intercept using simple y = m*x + b formula
            slope = (y2-y1) / float(x2-x1)
            intercept = y1 - slope * x1
                        
            # reject slopes that are out of range (due to spurious lines in image)
            if abs(slope) > slope_high or abs(slope) < slope_low:
                continue
                
            # sort lines into right and left lanes by slope
            # remember y = 0 at top of image, so right lane has slope > 0
            if slope > 0:    
                right_slopes.append(slope)
                right_intercepts.append(intercept)
            else:
                left_slopes.append(slope)
                left_intercepts.append(intercept)
        
    # calculate average values of slopes and intercepts
    left_slope_average = sum(left_slopes) / float(len(left_slopes))
    left_intercept_average = sum(left_intercepts) / float(len(left_intercepts))
    right_slope_average = sum(right_slopes) / float(len(right_slopes))
    right_intercept_average = sum(right_intercepts) / float(len(right_intercepts))

    # calculate (x,y) coordinates for average lane lines
    # lines begin at bottom of image and go to the top of the ROI
    y_top = int( roi_h * y_max )
    y_bottom = y_max
    left_x_bottom = int( (y_bottom - left_intercept_average) / left_slope_average )
    left_x_top = int( (y_top - left_intercept_average) / left_slope_average )
    right_x_bottom = int( (y_bottom - right_intercept_average) / right_slope_average )
    right_x_top = int( (y_top - right_intercept_average) / right_slope_average )
        
    # draw average lane lines
    average_lines = [ [[left_x_bottom,y_bottom,left_x_top,y_top]], [[right_x_bottom,y_bottom,right_x_top,y_top]] ]
    average_im = np.zeros((im.shape[0], im.shape[1], 3), dtype=np.uint8)
    draw_lines(average_im,average_lines,thickness = 8)
 
    return average_im


In [4]:
# define main image-processing pipeline
    
def process_image(im, individual_lines=False, save=False):
    
    # grayscale image and apply gaussian smoothing
    gray = grayscale(im)
    smoothed_gray = gaussian_blur(gray,gaussian_kernel_size)
    
    # find edges with canny
    edges = canny(smoothed_gray,canny_low,canny_high)
    
    # create masked edges image with polygon mask
    y_max,x_max,_ = im.shape
    vertices = np.array([[ (0,y_max),(roi_w1*x_max,roi_h*y_max),
                         (roi_w2*x_max,roi_h*y_max),(x_max,y_max) ]],dtype=np.int32)
    masked_edges = region_of_interest(edges,vertices)
    
    # if looking for individual lines, use original hough helper function
    # if looking for extended lane lines, use modified hough helper function
    if individual_lines:
        lines = hough_lines(masked_edges, hough_rho, hough_theta, 
                        hough_threshold, hough_min_len, hough_min_gap)
    
    else:
        lines = hough_lines_extended(masked_edges, hough_rho, hough_theta, 
                        hough_threshold, hough_min_len, hough_min_gap, slope_high, slope_low)
 
    # combine lines image with original image and save if specified
    result = weighted_img(im, lines, α=0.8, β=1., λ=0.)
    if save:
        cv2.imwrite(os.path.join(output_dir,im_file),result)
        
    return result


In [5]:
# run lane line pipeline on each image in test dir to check if it works

for im_file in os.listdir(test_image_dir):
    
    im = cv2.imread(os.path.join(test_image_dir,im_file))
    process_image(im,individual_lines=True,save=True)
    

In [6]:
# now test on a video

white_output = 'test_videos_output/solidWhiteRight.mp4'
clip1 = VideoFileClip("test_videos/solidWhiteRight.mp4")
white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidWhiteRight.mp4
[MoviePy] Writing video test_videos_output/solidWhiteRight.mp4


100%|█████████▉| 221/222 [00:05<00:00, 44.07it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidWhiteRight.mp4 

CPU times: user 2.52 s, sys: 835 ms, total: 3.36 s
Wall time: 5.74 s


In [7]:
# now display the resulting video

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))

In [8]:
# now test on a harder video!
# this one has yellow lane lines in addition to white

yellow_output = 'test_videos_output/solidYellowLeft.mp4'
##clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4').subclip(0,5)
clip2 = VideoFileClip('test_videos/solidYellowLeft.mp4')
yellow_clip = clip2.fl_image(process_image)
%time yellow_clip.write_videofile(yellow_output, audio=False)

[MoviePy] >>>> Building video test_videos_output/solidYellowLeft.mp4
[MoviePy] Writing video test_videos_output/solidYellowLeft.mp4


100%|█████████▉| 681/682 [00:18<00:00, 27.54it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_videos_output/solidYellowLeft.mp4 

CPU times: user 8.22 s, sys: 2.55 s, total: 10.8 s
Wall time: 19.2 s


In [10]:
# now display the resulting video

HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(yellow_output))

In [11]:
# this is testing an alternative approach to creating ROI-masked image

def region_of_interest_modified(img, vertices):
    # masks all pixels within vertices to be zero
    
    # work with a copy of the input image
    masked_image = np.copy(img)
        
    # determine which color to fill the mask with depending on the input image channel number
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        mask_color = (0,) * channel_count
    else:
        mask_color = 0
 
    # fill pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(masked_image, vertices, mask_color)
    
    return masked_image

# run test

for im_file in os.listdir(test_image_dir):
    
    # load image
    im = cv2.imread(os.path.join(test_image_dir,im_file))
    
    # determine vertices
    y_max,x_max,_ = im.shape
    vertices = np.array([[ (0,0),(0,y_max),(roi_w1*x_max,roi_h*y_max),
                         (roi_w2*x_max,roi_h*y_max),(x_max,y_max),(x_max,0) ]],dtype=np.int32)

    result = region_of_interest_modified(im,vertices)

    cv2.imwrite(os.path.join(output_dir,'ROI-'+im_file),result)
